<a href="https://colab.research.google.com/github/karandomguy/PhishGuard/blob/dev/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install patool

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 2.7 MB/s eta 0:00:00


In [3]:
import patoolib as pl
pl.extract_archive("/content/gdrive/MyDrive/phishing_url_model.zip")

patool: Extracting /content/gdrive/MyDrive/phishing_url_model.zip ...
patool: running /usr/bin/7z x -o./Unpack_5zliufn1 -- /content/gdrive/MyDrive/phishing_url_model.zip
patool: ... /content/gdrive/MyDrive/phishing_url_model.zip extracted to `best_pipeline_url.pkl'.


'best_pipeline_url.pkl'

In [4]:
%cd /content/gdrive/MyDrive/
!cp best_pipeline_tfidf.pkl /content/


/content/gdrive/MyDrive


In [5]:
import re

def check_url(text):
    url_pattern = "[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)"
    urls = re.findall(url_pattern, text)
    return urls

In [6]:
from urllib.parse import urlparse
from googlesearch import search

def having_ip_address(url):
    match = re.search(
        '(([01]?\d\d?|2[0-4]\d|25[0-5])\.([01]?\d\d?|2[0-4]\d|25[0-5])\.([01]?\d\d?|2[0-4]\d|25[0-5])\.'
        '([01]?\d\d?|2[0-4]\d|25[0-5])\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\.(0x[0-9a-fA-F]{1,2})\.(0x[0-9a-fA-F]{1,2})\.(0x[0-9a-fA-F]{1,2})\/)' # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
    if match:
        # print match.group()
        return 1
    else:
        # print 'No matching pattern found'
        return 0

def abnormal_url(url):
    hostname = urlparse(url).hostname
    hostname = str(hostname)
    match = re.search(hostname, url)
    if match:
        # print match.group()
        return 1
    else:
        # print 'No matching pattern found'
        return 0

def google_index(url):
    site = search(url, 5)
    return 1 if site else 0

def count_dot(url):
    count_dot = url.count('.')
    return count_dot

def count_www(url):
    url.count('www')
    return url.count('www')

def count_atrate(url):
    return url.count('@')

def no_of_dir(url):
    urldir = urlparse(url).path
    return urldir.count('/')

def no_of_embed(url):
    urldir = urlparse(url).path
    return urldir.count('//')

def shortening_service(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    if match:
        return 1
    else:
        return 0

def count_https(url):
    return url.count('https')

def count_http(url):
    return url.count('http')

def count_per(url):
    return url.count('%')

def count_ques(url):
    return url.count('?')

def count_hyphen(url):
    return url.count('-')

def count_equal(url):
    return url.count('=')

def url_length(url):
    return len(str(url))

def hostname_length(url):
    return len(urlparse(url).netloc)

def suspicious_words(url):
    match = re.search('PayPal|login|signin|bank|account|update|free|lucky|service|bonus|ebayisapi|webscr',
                      url)
    if match:
        return 1
    else:
        return 0

def digit_count(url):
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits

def letter_count(url):
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return letters


In [7]:
!pip install tld

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.8/263.8 kB 4.5 MB/s eta 0:00:00


In [8]:
from urllib.parse import urlparse
from tld import get_tld
import os.path

def fd_length(url):
    urlpath= urlparse(url).path
    try:
        return len(urlpath.split('/')[1])
    except:
        return 0

def get_tld_length(url):
    try:
        tld = get_tld(url, fail_silently=True)
        return len(tld)
    except:
        return -1

In [9]:
def feature_extractor(url):
    features = []

    # Feature 1: Having IP Address
    ip_address_feature = having_ip_address(url)
    features.append(ip_address_feature)

    # Feature 2: Abnormal URL
    abnormal_url_feature=abnormal_url(url)
    features.append(abnormal_url_feature)

    # Feature 3: Count Dot
    dot_count_feature = count_dot(url)
    features.append(dot_count_feature)

    # Feature 4: Count www
    www_count_feature = count_www(url)
    features.append(www_count_feature)

    # Feature 5: Count @
    at_count_feature = count_atrate(url)
    features.append(at_count_feature)

    # Feature 6: Number of Directories
    dir_count_feature = no_of_dir(url)
    features.append(dir_count_feature)

    # Feature 7: Number of Embedded Directories
    embed_dir_count_feature = no_of_embed(url)
    features.append(embed_dir_count_feature)

    # Feature 8: Shortening Service
    shortening_service_feature = shortening_service(url)
    features.append(shortening_service_feature)

    # Feature 9: Count 'https'
    count_https_feature = count_https(url)
    features.append(count_https_feature)

    # Feature 10: Count 'http'
    count_http_feature = count_http(url)
    features.append(count_http_feature)

    # Feature 11: Count '%'
    count_per_feature = count_per(url)
    features.append(count_per_feature)

    # Feature 12: Count '?'
    count_ques_feature = count_ques(url)
    features.append(count_ques_feature)

    # Feature 13: Count '-'
    count_hyphen_feature = count_hyphen(url)
    features.append(count_hyphen_feature)

    # Feature 14: Count '='
    count_equal_feature = count_equal(url)
    features.append(count_equal_feature)

    # Feature 15: URL Length
    url_length_feature = url_length(url)
    features.append(url_length_feature)

    # Feature 16: Hostname Length
    hostname_length_feature = hostname_length(url)
    features.append(hostname_length_feature)

    # Feature 17: Suspicious Words
    suspicious_words_feature = suspicious_words(url)
    features.append(suspicious_words_feature)

    # Feature 18: Digit Count
    digit_count_feature = digit_count(url)
    features.append(digit_count_feature)

    # Feature 19: Letter Count
    letter_count_feature = letter_count(url)
    features.append(letter_count_feature)

    # Feature 20: FD Length
    fd_length_feature = fd_length(url)
    features.append(fd_length_feature)

    # Feature 21: TLD Length
    tld_length_feature = get_tld_length(url)
    features.append(tld_length_feature)

    return features

In [10]:
import joblib

def get_prediction(url, model_path):

    model = joblib.load(model_path)
    url_features = feature_extractor(url)
    prediction = model.predict([url_features])
    phishing=url+ ": Phishing"
    benign=url+ ": Benign"
    if prediction[0]==1:
      return phishing
    else:
      return benign
    #print("There is", i, "% chance that this URL is malicious!")


In [11]:
model_path = r"/content/best_pipeline_url.pkl"

def url_predict(url):
  prediction = get_prediction(url,model_path)
  print(prediction)

In [12]:
import joblib

best_pipeline = joblib.load('best_pipeline_tfidf.pkl')


In [13]:
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
email_pipeline = joblib.load('best_pipeline_tfidf.pkl')
def preprocess(text):
      words = word_tokenize(text)
      no_stop_words = [word for word in words if word.isalpha() and word.lower() not in stop_words]
      return ' '.join(no_stop_words)

def preprocess_email(email_text):

  # Preprocess the text
  preprocessed_email = preprocess(email_text)
  preprocessed_email=preprocessed_email.lower()
  return preprocessed_email
print(stop_words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


{'s', 'through', 'by', 'are', 'was', "don't", 'mustn', "should've", 'from', 'now', 'over', 'ours', "isn't", 'where', 'no', 'your', 'while', 'if', 'they', 'has', 'been', 'there', "it's", "won't", 'himself', 'any', 'should', 'few', 'ain', 'than', "doesn't", 'mightn', 'is', "wouldn't", 'during', 'him', "wasn't", "didn't", 'aren', 'how', 'which', 'when', 'ma', 'you', 'it', 'won', 'on', 'these', 'm', 're', 'me', 'does', 'under', 'shouldn', 'our', 'their', 'too', "mustn't", "she's", 'below', 'those', 'only', 'weren', "you've", "haven't", 'do', 'didn', "that'll", 'herself', 'yours', 'down', "shan't", 'of', 'did', 'he', 'can', 'that', "hasn't", 'itself', 'o', 'a', 'both', 'its', "hadn't", 'wasn', 't', 'haven', 'theirs', 'such', 'once', 'against', 'some', 'all', 'shan', 'doesn', 'am', 'themselves', 'off', 'own', 'for', 'so', 'hasn', 'in', "couldn't", 'an', 'whom', 'd', 'hers', 'because', 'then', 've', 'as', 'after', 'we', 'and', 'more', 'yourself', 'not', "weren't", 'don', 'until', 'between', "

In [14]:
def email_predict(email_text):
  predicted_label = email_pipeline.predict([preprocess_email(email_text)])

  # Print the predicted label (1 for Safe Email, 0 for Phishing Email)
  safe="Predicted Label: Safe Email"
  phishy="Predicted Label: Phishing Email"
  if predicted_label[0] == '1':
    return safe
  else:
    return phishy

In [15]:
def final(text):
  urls=check_url(text)
  if len(urls)==0:
    return email_predict(text)
  else:
    print(email_predict(text))
    print("URL Predictions")
    for i in urls:
      print(url_predict(i))

In [17]:
text=input()
print(final(text))

**Enroll in a Stanford Graduate Course**  No matter where you are, you can become a Stanford student by enrolling in an [online graduate course](https://eventing.coursera.org/redirectSigned/eyJrZXkiOiJlbWFpbC5saW5rLm9wZW4iLCJ2YWx1ZSI6eyJ1cmwiOiJodHRwczovL29ubGluZS5zdGFuZm9yZC5lZHUvd2hhdC15b3UtbmVlZC1rbm93LWFib3V0LW9ubGluZS1ncmFkdWF0ZS1jb3Vyc2VzP3V0bV9zb3VyY2U9Q291cnNlcmEmdXRtX21lZGl1bT1lbWFpbCZ1dG1fY2FtcGFpZ249V2ludGVyMjRfR3JhZEVucm9sbG1lbnQiLCJ0cmFja2luZyI6eyJ1c2VySWQiOjk1MTc3NDYzLCJ1c2VyRW1haWwiOiJrYXJhbmIxMzE0QGdtYWlsLmNvbSIsIm5vdGlmaWNhdGlvblR5cGUiOiJvcGVuY291cnNlLnRhcmdldGVkbWVzc2FnZXMubWFya2V0aW5nIiwiY2FtcGFpZ25JZCI6InBhcnRuZXIuMS5ib2lGS3RyZVR4YXZ2YTRYV1dDdzh3IiwiY2FtcGFpZ25Hcm91cCI6InBhcnRuZXIuMS5vcGVuY291cnNlLnRhcmdldGVkbWVzc2FnZXMubWFya2V0aW5nIiwibGlua3MiOltdfX0sInVzZXJJZCI6OTUxNzc0NjN9.PA-FbUAR44_u0bgoUzTIbSpqCtS4YDDKX4WZsssyUaI). Join thousands of learners from around the world who have advanced their careers and accomplished their goals with courses for credit from Stanford

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


eventing.coursera.org/redirectSigned/eyJrZXkiOiJlbWFpbC5saW5rLm9wZW4iLCJ2YWx1ZSI6eyJ1cmwiOiJodHRwczovL29ubGluZS5zdGFuZm9yZC5lZHUvd2hhdC15b3UtbmVlZC1rbm93LWFib3V0LW9ubGluZS1ncmFkdWF0ZS1jb3Vyc2VzP3V0bV9zb3VyY2U9Q291cnNlcmEmdXRtX21lZGl1bT1lbWFpbCZ1dG1fY2FtcGFpZ249V2ludGVyMjRfR3JhZEVucm9sbG1lbnQiLCJ0cmFja2luZyI6eyJ1c2VySWQiOjk1MTc3NDYzLCJ1c2VyRW1haWwiOiJrYXJhbmIxMzE0QGdtYWlsLmNvbSIsIm5vdGlmaWNhdGlvblR5cGUiOiJvcGVuY291cnNlLnRhcmdldGVkbWVzc2FnZXMubWFya2V0aW5nIiwiY2FtcGFpZ25JZCI6InBhcnRuZXIuMS5ib2lGS3RyZVR4YXZ2YTRYV1dDdzh3IiwiY2FtcGFpZ25Hcm91cCI6InBhcnRuZXIuMS5vcGVuY291cnNlLnRhcmdldGVkbWVzc2FnZXMubWFya2V0aW5nIiwibGlua3MiOltdfX0sInVzZXJJZCI6OTUxNzc0NjN9.PA-FbUAR44_u0bgoUzTIbSpqCtS4YDDKX4WZsssyUaI).: Benign
None


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


eventing.coursera.org/redirectSigned/eyJrZXkiOiJlbWFpbC5saW5rLm9wZW4iLCJ2YWx1ZSI6eyJ1cmwiOiJodHRwczovL3d3dy55b3V0dWJlLmNvbS9Ac3RhbmZvcmRvbmxpbmUvcGxheWxpc3RzP3ZpZXc9NTAmc29ydD1kZCZzaGVsZl9pZD0zP3V0bV9zb3VyY2UlM0RDb3Vyc2VyYSZ1dG1fbWVkaXVtPWVtYWlsJnV0bV9jYW1wYWlnbj1XaW50ZXIyNF9HcmFkRW5yb2xsbWVudCZ1dG1fc291cmNlPW90aGVyIiwidHJhY2tpbmciOnsidXNlcklkIjo5NTE3NzQ2MywidXNlckVtYWlsIjoia2FyYW5iMTMxNEBnbWFpbC5jb20iLCJub3RpZmljYXRpb25UeXBlIjoib3BlbmNvdXJzZS50YXJnZXRlZG1lc3NhZ2VzLm1hcmtldGluZyIsImNhbXBhaWduSWQiOiJwYXJ0bmVyLjEuYm9pRkt0cmVUeGF2dmE0WFdXQ3c4dyIsImNhbXBhaWduR3JvdXAiOiJwYXJ0bmVyLjEub3BlbmNvdXJzZS50YXJnZXRlZG1lc3NhZ2VzLm1hcmtldGluZyIsImxpbmtzIjpbXX19LCJ1c2VySWQiOjk1MTc3NDYzfQ.oAVFT1uRXTA_-LhTMG1BTSL3xiIJ9Z77vaa3vJ4xdyQ).: Benign
None


...

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


eventing.coursera.org/redirectSigned/eyJrZXkiOiJlbWFpbC5saW5rLm9wZW4iLCJ2YWx1ZSI6eyJ1cmwiOiJodHRwczovL2xlYXJuLnN0YW5mb3JkLmVkdS9ncmFkdWF0ZS1jb3Vyc2VzLUxQLmh0bWw_dXRtX3NvdXJjZT1Db3Vyc2VyYSZ1dG1fbWVkaXVtPWVtYWlsJnV0bV9jYW1wYWlnbj1XaW50ZXIyNF9HcmFkRW5yb2xsbWVudCIsInRyYWNraW5nIjp7InVzZXJJZCI6OTUxNzc0NjMsInVzZXJFbWFpbCI6ImthcmFuYjEzMTRAZ21haWwuY29tIiwibm90aWZpY2F0aW9uVHlwZSI6Im9wZW5jb3Vyc2UudGFyZ2V0ZWRtZXNzYWdlcy5tYXJrZXRpbmciLCJjYW1wYWlnbklkIjoicGFydG5lci4xLmJvaUZLdHJlVHhhdnZhNFhXV0N3OHciLCJjYW1wYWlnbkdyb3VwIjoicGFydG5lci4xLm9wZW5jb3Vyc2UudGFyZ2V0ZWRtZXNzYWdlcy5tYXJrZXRpbmciLCJsaW5rcyI6W119fSwidXNlcklkIjo5NTE3NzQ2M30.unVHUrLkeptUMQKQ6yLfzP2WrEhF77wrz_5cScnv_eQ): Benign
None


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


eventing.coursera.org/redirectSigned/eyJrZXkiOiJlbWFpbC5saW5rLm9wZW4iLCJ2YWx1ZSI6eyJ1cmwiOiJodHRwczovL29ubGluZS5zdGFuZm9yZC5lZHUvP3V0bV9zb3VyY2U9Q291cnNlcmEmdXRtX21lZGl1bT1lbWFpbCZ1dG1fY2FtcGFpZ249V2ludGVyMjRfR3JhZEVucm9sbG1lbnQiLCJ0cmFja2luZyI6eyJ1c2VySWQiOjk1MTc3NDYzLCJ1c2VyRW1haWwiOiJrYXJhbmIxMzE0QGdtYWlsLmNvbSIsIm5vdGlmaWNhdGlvblR5cGUiOiJvcGVuY291cnNlLnRhcmdldGVkbWVzc2FnZXMubWFya2V0aW5nIiwiY2FtcGFpZ25JZCI6InBhcnRuZXIuMS5ib2lGS3RyZVR4YXZ2YTRYV1dDdzh3IiwiY2FtcGFpZ25Hcm91cCI6InBhcnRuZXIuMS5vcGVuY291cnNlLnRhcmdldGVkbWVzc2FnZXMubWFya2V0aW5nIiwibGlua3MiOltdfX0sInVzZXJJZCI6OTUxNzc0NjN9.3yeGAnZ75-k63h16MNeppR9gzbodoiU1L1sur0gnSEU): Benign
None
eventing.coursera.org/redirectSigned/eyJrZXkiOiJlbWFpbC5saW5rLm9wZW4iLCJ2YWx1ZSI6eyJ1cmwiOiJodHRwczovL3NjcGQuc3RhbmZvcmQuZWR1Lz91dG1fc291cmNlPUNvdXJzZXJhJnV0bV9tZWRpdW09ZW1haWwmdXRtX2NhbXBhaWduPVdpbnRlcjI0X0dyYWRFbnJvbGxtZW50IiwidHJhY2tpbmciOnsidXNlcklkIjo5NTE3NzQ2MywidXNlckVtYWlsIjoia2FyYW5iMTMxNEBnbWFpbC5jb20iLCJub3RpZmljYXRpb25UeXBlIjoib3BlbmNvdXJ

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
